<a href="https://colab.research.google.com/github/ikfaisal/dsia/blob/master/DSIA_Lab_2_1_3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 2.1.3: 
# *Database Programming with Python and SQLite*

<a name="demo"></a>
## Using SQLite

### Advantages of SQLite

- does not run on a separate server
- creates portable SQL databases saved in a single file
- databases are stored in a very efficient manner and allow fast querying
- ideal for small databases or databases that need to be copied across machines.
- prototyping applications (e.g. as an embedded database server in a Python program).



### The `sqlite3` Command Line Utility

- useful for basic SQL tasks and databse maintenance
- for creating and dropping databases, it may be safer to use the command line than to roll the code into a Python program

1. Add the sqlite3 installation folder to your PATH environment variable.
2. Open a command window.
3. Navigate to your preferred working directory.
4. Start the sqlite3 command line utility and create a database called "ex1":

`$ sqlite3 ex1`

Output:  

`SQLite version 3.8.5 2014-05-29 12:36:14
Enter ".help" for usage hints.
sqlite>`

*Nb. If you leave out the database name (after `sqlite3`, above) a new, temporary database will be created; it will be destroyed upon exiting sqlite3.*  


Enter the following commands at the sqlite prompt (not shown):

`create table tbl1(one varchar(10), two smallint);
insert into tbl1 values('hello!',10);
insert into tbl1 values('goodbye', 20);
select * from tbl1;`

Output:  

`hello!|10
goodbye|20`

Enter `.help` at the sqlite3 command prompt. This lists the available "dot commands". 

Some of the more useful dot commands include:

`  
.backup ?DB? FILE      Backup DB (default "main") to FILE.
.cd DIRECTORY          Change the working directory to DIRECTORY
.clone NEWDB           Clone data into NEWDB from the existing database
.databases             List names and files of attached databases
.dump ?TABLE? ...      Dump the database in an SQL text format.
.excel                 Display the output of next command in a spreadsheet
.exit                  Exit this program
.headers on|off        Turn display of headers on or off
.help                  Show this message
.import FILE TABLE     Import data from FILE into TABLE
.indexes ?TABLE?       Show names of all indexes.
.quit                  Exit this program
.read FILENAME         Execute SQL in FILENAME
.restore ?DB? FILE     Restore content of DB (default "main") from FILE
.save FILE             Write in-memory database into FILE
.tables ?TABLE?        List names of tables.`


### The `sqlite3` package

The easiest way to incorporate an SQL database into a Python application is by using the `sqlite3` package for [`Python 2.7`](https://docs.python.org/2.7/library/sqlite3.html) or [`Python 3.x`](https://docs.python.org/3/library/sqlite3.html).

Open a connection to an SQLite database file.  As before, if the file does not already exist it will automatically be created.

In [1]:
import sqlite3
sqlite_db = 'test_db.sqlite'
conn = sqlite3.connect(sqlite_db) 
c = conn.cursor()

Commands can be executed by passing them as string arguments to the `execute` method of the cursor we just created for this database.

*Nb. If this script has never been run before, the following will raise an (inconsequential) error message:*

In [2]:
c.execute('DROP TABLE houses;')

# Save (commit) the changes:
conn.commit()

In [3]:
c.execute('CREATE TABLE houses (field1 INTEGER PRIMARY KEY, sqft INTEGER, bdrms INTEGER, age INTEGER, price INTEGER);')
conn.commit()

With the database saved the table should now be viewable using SQLite Manager.

#### Adding data

Since we're back in python, we can now use regular programming techniques in conjunction with the sqlite connection.  In particular, the cursor's `execute()` method supports value substitution using the `?` character, which makes adding multiple records a bit easier.  See the [docs](https://docs.python.org/2.7/library/sqlite3.html) for more details.

> c.execute(sql_command, values)

In [4]:
# A
last_sale = (None, 4000, 5, 22, 619000)
# Execute Command
c.execute('INSERT INTO houses VALUES(?,?,?,?,?)',last_sale)
# Remember to commit the changes
conn.commit()

Notice that in this syntax we use the python `None` value, rather than `NULL`, to trigger SQLite to auto-increment the Primary Key. 

There is a related cursor method `executemany()` which takes an array of tuples and loops through them, substituting one tuple at a time.

> c.executemany(sql_command, values)

In [5]:
# A
recent_sales = [
  (None, 2390, 4, 34, 319000),
  (None, 1870, 3, 14, 289000),
  (None, 1505, 3, 90, 269000),
]

# Nb. The "?" parameters are placeholders for data that will map to the table columns during insertion;
# this is a security measure against SQL injection attacks:
# 
c.executemany('INSERT INTO houses VALUES(?,?,?,?,?)',recent_sales)
conn.commit()

Select all rows from houses

In [6]:
# 
for row in c.execute('SELECT * FROM houses'):
    print (row)

(1, 4000, 5, 22, 619000)
(2, 2390, 4, 34, 319000)
(3, 1870, 3, 14, 289000)
(4, 1505, 3, 90, 269000)





#### Adding data from a csv file
One way to populate the database from a file is to use `numpy.genfromtxt` to read the file into an array (converted to a list for easier handling), and then `INSERT` those records into the database.  The `genfromtxt` function has options including the output data type, handling of missing values, skipping of header and footer rows, columns to read, and more. 

In [7]:
from numpy import genfromtxt

# import into nparray of ints, then convert to list of lists:
data = genfromtxt('../../DATA/housing-data.csv', dtype = 'i8', 
                    delimiter = ',', skip_header = 1).tolist()

Suppose we need to put a placeholder in the first column for data that will be available later. Best practice is to insert the value `None`:

In [8]:
# prepend a None value to beginning of each sub-list:
for d in data:
    d.insert(0, None)

In [9]:
print(data)

[[None, 2104, 3, 70, 399900], [None, 1600, 3, 28, 329900], [None, 2400, 3, 44, 369000], [None, 1416, 2, 49, 232000], [None, 3000, 4, 75, 539900], [None, 1985, 4, 61, 299900], [None, 1534, 3, 12, 314900], [None, 1427, 3, 57, 198999], [None, 1380, 3, 14, 212000], [None, 1494, 3, 15, 242500], [None, 1940, 4, 7, 239999], [None, 2000, 3, 27, 347000], [None, 1890, 3, 45, 329999], [None, 4478, 5, 49, 699900], [None, 1268, 3, 58, 259900], [None, 2300, 4, 77, 449900], [None, 1320, 2, 62, 299900], [None, 1236, 3, 78, 199900], [None, 2609, 4, 5, 499998], [None, 3031, 4, 21, 599000], [None, 1767, 3, 44, 252900], [None, 1888, 2, 79, 255000], [None, 1604, 3, 13, 242900], [None, 1962, 4, 53, 259900], [None, 3890, 3, 36, 573900], [None, 1100, 3, 60, 249900], [None, 1458, 3, 29, 464500], [None, 2526, 3, 13, 469000], [None, 2200, 3, 28, 475000], [None, 2637, 3, 25, 299900], [None, 1839, 2, 40, 349900], [None, 1000, 1, 5, 169900], [None, 2040, 4, 75, 314900], [None, 3137, 3, 67, 579900], [None, 1811, 4, 

*Nb. This is why we converted the input array to a list. An array can only hold one type of data (integers in this case) so we could not have inserted `None` before we did this conversion.*

In [10]:
# check:
print(type(data))
print(data[0:3])

<class 'list'>
[[None, 2104, 3, 70, 399900], [None, 1600, 3, 28, 329900], [None, 2400, 3, 44, 369000]]


Now we can insert each list item as a row of fields in the database.

In [11]:
# loop through data, running an INSERT on each record (i.e. sublist):
for d in data:
    c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d)

conn.commit()

In this case, because we were inserting the same value for all records, so we could have simply used a 'None' in the numpy `insert` method at column 0.

In [12]:
import numpy as np
np.asarray([1200, 3, 15, 250000])

array([  1200,      3,     15, 250000])

In [13]:
import numpy as np
d1 = np.asarray([1200, 3, 15, 250000])
d1 = d1.tolist()
d1.insert(0, None)
d1
c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d1)
conn.commit()

In [14]:
print(data)

[[None, 2104, 3, 70, 399900], [None, 1600, 3, 28, 329900], [None, 2400, 3, 44, 369000], [None, 1416, 2, 49, 232000], [None, 3000, 4, 75, 539900], [None, 1985, 4, 61, 299900], [None, 1534, 3, 12, 314900], [None, 1427, 3, 57, 198999], [None, 1380, 3, 14, 212000], [None, 1494, 3, 15, 242500], [None, 1940, 4, 7, 239999], [None, 2000, 3, 27, 347000], [None, 1890, 3, 45, 329999], [None, 4478, 5, 49, 699900], [None, 1268, 3, 58, 259900], [None, 2300, 4, 77, 449900], [None, 1320, 2, 62, 299900], [None, 1236, 3, 78, 199900], [None, 2609, 4, 5, 499998], [None, 3031, 4, 21, 599000], [None, 1767, 3, 44, 252900], [None, 1888, 2, 79, 255000], [None, 1604, 3, 13, 242900], [None, 1962, 4, 53, 259900], [None, 3890, 3, 36, 573900], [None, 1100, 3, 60, 249900], [None, 1458, 3, 29, 464500], [None, 2526, 3, 13, 469000], [None, 2200, 3, 28, 475000], [None, 2637, 3, 25, 299900], [None, 1839, 2, 40, 349900], [None, 1000, 1, 5, 169900], [None, 2040, 4, 75, 314900], [None, 3137, 3, 67, 579900], [None, 1811, 4, 

#### Deleting Rows

The `DELETE FROM` statement can be used with a `WHERE` clause to specify rows to delete based on some criteria.

In [15]:
c.execute('DELETE FROM houses WHERE field1 IN (52, 53)')
conn.commit()

In [16]:
import pandas as pd
pd.read_sql_query('select * from houses WHERE field1 IN (52, 53)',conn)

,field1,sqft,bdrms,age,price


QUIZ: What would `DELETE FROM houses` do?

#### Filtering Rows

**1. Select Rows Where Bedrooms = 4**

In [17]:
# ANSWER
bed4 = c.execute('SELECT * FROM houses WHERE bdrms = 4')
conn.commit()
bed4.fetchall()

[(2, 2390, 4, 34, 319000),
 (9, 3000, 4, 75, 539900),
 (10, 1985, 4, 61, 299900),
 (15, 1940, 4, 7, 239999),
 (20, 2300, 4, 77, 449900),
 (23, 2609, 4, 5, 499998),
 (24, 3031, 4, 21, 599000),
 (28, 1962, 4, 53, 259900),
 (37, 2040, 4, 75, 314900),
 (39, 1811, 4, 24, 285900),
 (42, 2132, 4, 28, 345000),
 (43, 4215, 4, 66, 549000),
 (44, 2162, 4, 43, 287000),
 (47, 2567, 4, 57, 314000),
 (50, 1852, 4, 64, 299900)]

**2. Run a query to calculate the average floor area and price of each size of house (i.e. by number of bedrooms):**

In [20]:
# ANSWER
results = c.execute("SELECT bdrms, AVG(sqft) AS avg_sqft, AVG(price) AS avg_price FROM houses GROUP BY bdrms")
results.fetchall()

[(1, 1000.0, 169900.0),
 (2, 1496.5, 280866.6666666667),
 (3, 1827.5555555555557, 322892.51851851854),
 (4, 2399.733333333333, 373553.13333333336),
 (5, 4239.0, 659450.0)]

### Pandas connector

While databases provide many analytical capabilities, at some point we may need to pull data into Python for more flexible processing. Large, fixed operations would be more efficient in a database, but Pandas allows for interactive processing.

For example, if you want to aggregate nightly log-ins or sales for a report or dashboard, this would be a fixed operation on a large dataset. These computations would run more efficiently in the database system itself.

However, if we wanted to model the patterns of login behaviour or factors driving sales, then we would import the data to Python where we could use its simple interfaces to powerful analytic libraries.

In [21]:
import pandas as pd
from pandas.io import sql

Pandas can connect to most relational databases. In this demonstration, we will create and connect to a SQLite database.

### Writing data into a database

Data in Pandas can be loaded into a relational database. 

If the data table is not too large, we can load all of it into a Pandas DataFrame:

In [22]:
# Nb. Use low_memory=False to ensure that type inference does not fail due to buffered processing of input:
data = pd.read_csv('../../DATA/housing-data.csv', low_memory = False)
data.head()

,sqft,bdrms,age,price
0,2104,3,70,399900
1,1600,3,28,329900
2,2400,3,44,369000
3,1416,2,49,232000
4,3000,4,75,539900


We can move data in the opposite direction -- from a DataFrame to a database -- using the `to_sql` command, similar to the `to_csv` command.

`to_sql` takes as arguments:
    - `name`, the table name to create
    - `con`, a connection to a database
    - `index`, whether to input the index column
    - `schema`, if we want to write a custom schema for the new table
    - `if_exists`, what to do if the table already exists. We can overwrite it, add to it, or fail

This copies our `data` DataFrame to a sqlite3 table called `houses_pandas`:

In [23]:
data.to_sql('houses_pandas', con = conn, if_exists = 'replace', index = False)

Run a query to get the average price of each house size from this table:

In [24]:
print(c)

In [25]:
results = c.execute("SELECT bdrms, AVG(sqft) AS avg_sqft FROM houses GROUP BY bdrms")
results.fetchall()

[(1, 1000.0),
 (2, 1496.5),
 (3, 1827.5555555555557),
 (4, 2399.733333333333),
 (5, 4239.0)]

In [26]:
#ANSWER
data.groupby('bdrms')['sqft'].mean()

bdrms
1    1000.000000
2    1496.500000
3    1838.760000
4    2400.428571
5    4478.000000
Name: sqft, dtype: float64

## Discussion

**Scenarios for using Pandas with SQLite**

1. When would you want to use Pandas on a dataset before storing it in a database?

2. When would you want to use Pandas on a dataset retrieved from a database?

<a name="guided-practice"></a>


# Reference: SQL Syntax 


#### SELECT Statement
Every query should start with `SELECT`.  `SELECT` is followed by the names of the columns in the output.

`SELECT` is always paired with `FROM`, and `FROM` identifies the table to retrieve data from.

```sql
SELECT
<columns>
FROM
<table>
```

`SELECT *` denotes returns *all* of the columns.

Housing Data example:
```sql
SELECT
sqft, bdrms
FROM houses_pandas;
```

**Check:** Write a query that returns the `sqft`, `bdrms` and `price`.

```sql
SELECT
sqft, bdrms, price
FROM houses_pandas;
```

#### WHERE Clause
`WHERE` is used to filter table to a specific criteria and follows the `FROM` clause.

```sql
SELECT
<columns>
FROM
<table>
WHERE
<condition>
```
Example:
```sql
SELECT
sqft, bdrms, age, price
FROM houses_pandas
WHERE bdrms = 2 and price < 250000;
```

The condition is effectively a row filter; rows that match the condition will be included in the rowset that is returned by the query.

**Check:** Write a query that returns the `sqft`, `bdrms`, `age` for houses older than 60 years.
>```sql
SELECT
sqft, bdrms, age
FROM houses_pandas
WHERE age > 60;
```

### AGGREGATIONS

Aggregations (or aggregate functions) are functions where the values of multiple rows are grouped together as input on certain criteria to form a single value of more significant meaning or measurement such as a set, a bag or a list.

Examples of aggregate funtions:

- Average (i.e., arithmetic mean)
- Count
- Maximum
- Minimum
- Median
- Mode
- Sum

In SQL they are performed in a `SELECT` statement as follows.

```sql
SELECT COUNT(price)
FROM houses_pandas;
```

```sql
SELECT AVG(sqft), MIN(price), MAX(price)
FROM houses_pandas
WHERE bdrms = 2;
```

### Read Order Data
- P12-ListOfOrders
- P12-OrderBreakdown

#### 1. Read CSV into DataFrame

In [27]:
import pandas as pd
import sqlite3
from pandas.io import sql

In [28]:
# Reading CSV to Dataframe
orders = pd.read_csv('../../DATA/P12-ListOfOrders.csv', encoding = 'utf-8')
orders_break_down =  pd.read_csv('../../DATA/P12-OrderBreakdown.csv', encoding = 'utf-8')

In [29]:
orders.head()

,Order ID,Order Date,Customer Name,Country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France


In [30]:
orders_break_down.head()

,Order ID,Product Name,Discount,Sales,Quantity,Category
0,IT-2011-3647632,"Enermax Note Cards, Premium",0.5,45.0,3,Office Supplies
1,ES-2011-4869686,"Dania Corner Shelving, Traditional",0.0,854.0,7,Furniture
2,ES-2011-4939443,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,3,Office Supplies
3,IT-2011-2942451,"Boston Markers, Easy-Erase",0.5,27.0,2,Office Supplies
4,IT-2011-2942451,"Eldon Folders, Single Width",0.5,17.0,2,Office Supplies


#### 2. Replace Space with Underscore in Column Names

In [31]:
# ANSWER
orders.columns = [o.replace(' ', '_') for o in orders.columns.str.lower()]
orders.columns

Index(['order_id', 'order_date', 'customer_name', 'country'], dtype='object')

In [32]:
# ANSWER
orders_break_down.columns = [o.replace(' ', '_') for o in orders_break_down.columns.str.lower()]

#### 3. Check DataTypes 

In [33]:
# ANSWER
orders.dtypes

order_id         object
order_date       object
customer_name    object
country          object
dtype: object

In [34]:
# ANSWER
orders_break_down.dtypes

order_id         object
product_name     object
discount        float64
sales           float64
quantity          int64
category         object
dtype: object

#### 4. Save these two dataframes as a table in sqlite

In [48]:
# Establishing Local DB connection #get con
db_connection = sqlite3.connect('../../DATA/eshop.db.sqlite')
# Save these two dataframes as a table in sqlite
#c = conn.cursor()
orders.to_sql('orders', con= db_connection, if_exists='replace', index = False)
orders_break_down.to_sql('orders_break_down', con = db_connection, if_exists = 'replace', index = False)

#### 5. Select Number of Orders for Each Customer

We can use the `sql` function in `pandas.io` to run SQL queries to the database with the `.read_sql()` method. Here's an example:

In [50]:
pd.read_sql_query('select * from orders_break_down;',con=db_connection)

,order_id,product_name,discount,sales,quantity,category
0,IT-2011-3647632,"Enermax Note Cards, Premium",0.5,45.0,3,Office Supplies
1,ES-2011-4869686,"Dania Corner Shelving, Traditional",0.0,854.0,7,Furniture
2,ES-2011-4939443,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,3,Office Supplies
3,IT-2011-2942451,"Boston Markers, Easy-Erase",0.5,27.0,2,Office Supplies
4,IT-2011-2942451,"Eldon Folders, Single Width",0.5,17.0,2,Office Supplies
...,...,...,...,...,...,...
8042,ES-2014-3458802,"Ikea Stackable Bookrack, Traditional",0.0,245.0,2,Furniture
8043,IT-2014-3715679,"BIC Pencil Sharpener, Fluorescent",0.5,30.0,2,Office Supplies
8044,IT-2014-3715679,"Avery Binder Covers, Recycled",0.5,23.0,4,Office Supplies
8045,IT-2014-3715679,"StarTech Phone, Red",0.5,108.0,3,Technology


In [40]:
# Select first 10 rows of all columns in orders table

query = """
        SELECT * 
        FROM orders 
        LIMIT 10;
        """

orders_head_10 = sql.read_sql(query, con=db_connection)
orders_head_10

,order_id,order_date,customer_name,country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France
5,ES-2011-5433855,2011-01-07,Dave Poirier,France
6,IT-2011-4546695,2011-01-08,Darren Powers,France
7,ES-2011-1138719,2011-01-11,Eric Murdock,Italy
8,ES-2011-1466305,2011-01-11,Mick Brown,Austria
9,ES-2011-4359424,2011-01-11,Dorothy Dickinson,Spain


In [42]:
# Select Number of Orders for Each Customer

query = """
        SELECT customer_name, COUNT(order_id) as No_orders 
        FROM orders 
        GROUP BY customer_name
        order by COUNT(order_id) desc
        ;
        """

orders_customer = sql.read_sql(query, con=db_connection)
orders_customer

,customer_name,No_orders
0,John Grady,13
1,Joel Jenkins,12
2,Aaron Smayling,12
3,Yoseph Carroll,11
4,Sarah Brown,11
...,...,...
787,Brooke Gillingham,1
788,Barbara Fisher,1
789,Art Ferguson,1
790,Andy Reiter,1


#### 6. Select Number of Customers for Each Country

In [43]:
# Select Number of Customers for Each Country
query = """
        SELECT country, count(customer_name) as No_customers
        FROM orders
        GROUP BY country
        ORDER BY count(customer_name)
        ;
        """
country_cust = sql.read_sql(query, con= db_connection)
country_cust

,country,No_customers
0,Denmark,29
1,Finland,34
2,Norway,37
3,Portugal,37
4,Switzerland,40
5,Ireland,50
6,Belgium,68
7,Sweden,100
8,Austria,135
9,Netherlands,194


#### 7.A Select discount, sales, quantity for Each Order from orders_break_down Table

In [51]:
#  Select discount, sales, quantity for Each Order from orders_break_down Table
query = """
        select order_id, discount, sales, quantity
        from orders_break_down
        ;
        """
orders_breakdown = sql.read_sql(query, con=db_connection)
orders_breakdown
        

,order_id,discount,sales,quantity
0,IT-2011-3647632,0.5,45.0,3
1,ES-2011-4869686,0.0,854.0,7
2,ES-2011-4939443,0.0,140.0,3
3,IT-2011-2942451,0.5,27.0,2
4,IT-2011-2942451,0.5,17.0,2
...,...,...,...,...
8042,ES-2014-3458802,0.0,245.0,2
8043,IT-2014-3715679,0.5,30.0,2
8044,IT-2014-3715679,0.5,23.0,4
8045,IT-2014-3715679,0.5,108.0,3


#### 7.B Select discount, sales, quantity, total price for Each Order from orders_break_down Table

> Total Price = sales * quantity - discount

In [53]:
# Select discount, sales, quantity, total price for Each Order from orders_break_down Table
query = """
        select order_id, discount, sales, quantity, (sales*quantity)-discount as total_price
        from orders_break_down
        ;
        """
orders_breakdown = sql.read_sql(query, con=db_connection)
orders_breakdown
        

,order_id,discount,sales,quantity,total_price
0,IT-2011-3647632,0.5,45.0,3,134.5
1,ES-2011-4869686,0.0,854.0,7,5978.0
2,ES-2011-4939443,0.0,140.0,3,420.0
3,IT-2011-2942451,0.5,27.0,2,53.5
4,IT-2011-2942451,0.5,17.0,2,33.5
...,...,...,...,...,...
8042,ES-2014-3458802,0.0,245.0,2,490.0
8043,IT-2014-3715679,0.5,30.0,2,59.5
8044,IT-2014-3715679,0.5,23.0,4,91.5
8045,IT-2014-3715679,0.5,108.0,3,323.5


#### 7.C Select All Orders from orders_break_down Table Where Total Price Greater Than 100

In [58]:
# Select All Orders from orders_break_down Table Where Total Price Greater Than 100
query = """
        select *, (sales*quantity)-discount as total_price
        from orders_break_down
        where total_price > 100
        order by total_price asc
        ;
        """
p100= sql.read_sql(query, con=db_connection)
p100
        

,order_id,product_name,discount,sales,quantity,category,total_price
0,IT-2011-2668777,"Binney & Smith Canvas, Blue",0.5,51.0,2,Office Supplies,101.5
1,IT-2011-1165265,"Cardinal Binding Machine, Durable",0.5,51.0,2,Office Supplies,101.5
2,ES-2012-1763830,"Elite Scissors, High Speed",0.5,34.0,3,Office Supplies,101.5
3,IT-2012-4099397,"Sanford Canvas, Blue",0.5,51.0,2,Office Supplies,101.5
4,ES-2013-4250046,"Cardinal Binding Machine, Durable",0.5,51.0,2,Office Supplies,101.5
...,...,...,...,...,...,...,...
6461,ES-2014-2637201,"Brother Fax Machine, Laser",0.0,4141.0,13,Technology,53833.0
6462,IT-2013-4602742,"Samsung Smart Phone, VoIP",0.1,5726.0,10,Technology,57259.9
6463,ES-2012-2183106,"Canon Wireless Fax, High-Speed",0.1,4748.0,14,Technology,66471.9
6464,ES-2011-3248922,"Hamilton Beach Stove, Silver",0.0,6517.0,12,Office Supplies,78204.0


#### 8. Select All Customer And The Product They Have Bought

In [62]:
# Select All Customer And The Product They Have Bought
query = """
        select customer_name, product_name
        from orders inner join orders_break_down
        on orders.order_id = orders_break_down.order_id
        order by customer_name
        ;
        """
customer_product= sql.read_sql(query, con=db_connection)
customer_product
        

,customer_name,product_name
0,Aaron Bergman,"Apple Office Telephone, Cordless"
1,Aaron Bergman,"Elite Box Cutter, High Speed"
2,Aaron Bergman,"Harbour Creations Round Labels, Laser Printer ..."
3,Aaron Bergman,"Binney & Smith Markers, Fluorescent"
4,Aaron Bergman,"Novimex Shipping Labels, Alphabetical"
...,...,...
8042,Zuschuss Donatelli,"Hewlett Fax Machine, Laser"
8043,Zuschuss Donatelli,"Office Star Executive Leather Armchair, Adjust..."
8044,Zuschuss Donatelli,"Accos Rubber Bands, Metal"
8045,Zuschuss Donatelli,"Memorex Numeric Keypad, Bluetooth"


#### 9.A Select Number of 'Furniture' Orders For Each Country 

In [72]:
# Select Number of 'Furniture' Orders For Each Country
query = """
        select Country, category, COUNT(CATEGORY) AS No_Furniture_orders
        from orders inner join orders_break_down
        on orders.order_id = orders_break_down.order_id
        where category = 'Furniture'
        group by Country       
        order by No_Furniture_orders Desc
        ;
        """
furniture= sql.read_sql(query, con=db_connection)
furniture

,country,category,No_Furniture_orders
0,France,Furniture,299
1,Germany,Furniture,264
2,United Kingdom,Furniture,180
3,Italy,Furniture,136
4,Spain,Furniture,123
5,Netherlands,Furniture,76
6,Austria,Furniture,40
7,Sweden,Furniture,36
8,Belgium,Furniture,20
9,Finland,Furniture,16


#### 9.B Select Number of 'Furniture' Orders For The Country Denmark

In [81]:
# Select Number of 'Furniture' Orders For The Country Denmark
query = """
        select Country, category, COUNT(CATEGORY) AS No_Furniture_orders
        from orders inner join orders_break_down
        on orders.order_id = orders_break_down.order_id
        where category = 'Furniture' and country = 'Denmark'
        group by Country   
        order by No_Furniture_orders Desc
        ;
        """
furnitureDenmark= sql.read_sql(query, con=db_connection)
furnitureDenmark

,country,category,No_Furniture_orders
0,Denmark,Furniture,9


#### 10. Select Total Sales With Discount and Without Discount for Each Country

In [98]:
query = """
            select Country, sum(sales) as total_sales_discount
            from orders inner join orders_break_down
            on orders.order_id = orders_break_down.order_id
            where discount > 0
            group by Country
        ;
        """
salesdiscount= sql.read_sql(query, con=db_connection)
salesdiscount

,country,total_sales_discount
0,Denmark,7763.0
1,France,479892.0
2,Germany,243257.0
3,Ireland,15998.0
4,Italy,78645.0
5,Netherlands,70313.0
6,Portugal,15106.0
7,Spain,84507.0
8,Sweden,30490.0
9,United Kingdom,89643.0


In [129]:
query = """
        WITH DISCOUNT (Country, sales_discount) AS (
        select Country, sum(sales)
        from orders O inner join orders_break_down OB
        on O.order_id = OB.order_id
        where discount > 0
        group by Country
        ),
        WITH NODISCOUNT (Country, sales_discount) AS (
        select Country, sum(sales)
        from orders O inner join orders_break_down OB
        on O.order_id = OB.order_id
        where discount <= 0
        group by Country
        )        
        SELECT D.Country, D.sales_discount, ND.SALES_DISCOUNT
        FROM DISCOUNT D INNER JOIN NODISCOUNT ND
        ON D.order_id =ND.order_id
        GROUP BY Country
        ;
        """
salesdics= sql.read_sql(query, con=db_connection)
salesdics

DatabaseError: Execution failed on sql '
        WITH DISCOUNT (Country, sales_discount) AS (
        select Country, sum(sales)
        from orders O inner join orders_break_down OB
        on O.order_id = OB.order_id
        where discount > 0
        group by Country
        ),
        WITH NODISCOUNT (Country, sales_discount) AS (
        select Country, sum(sales)
        from orders O inner join orders_break_down OB
        on O.order_id = OB.order_id
        where discount <= 0
        group by Country
        )        
        SELECT D.Country, D.sales_discount, ND.SALES_DISCOUNT
        FROM DISCOUNT D INNER JOIN NODISCOUNT ND
        ON D.order_id =ND.order_id
        GROUP BY Country
        ;
        ': near "NODISCOUNT": syntax error

In [125]:
# Select Total Sales With Discount and Without Discount for Each Country
query = """
        select a.country, a.sum(sales), b.sum(sales)
        From(select Country, sum(sales) as total_sales_discount
        from orders inner join orders_break_down
        on orders.order_id = orders_break_down.order_id
        where discount > 0
        group by Country)a
        JOIN(select Country, sum(sales) as total_sales_discount
        from orders inner join orders_break_down
        on orders.order_id = orders_break_down.order_id
        where discount <= 0
        group by Country)b
        ON a.Country=b.Country
        group by Country
        order a.Country Desc
        ;
        """
salesdics= sql.read_sql(query, con=db_connection)
salesdics

DatabaseError: Execution failed on sql '
        select a.country, a.sum(sales), b.sum(sales)
        From(select Country, sum(sales) as total_sales_discount
        from orders inner join orders_break_down
        on orders.order_id = orders_break_down.order_id
        where discount > 0
        group by Country)a
        JOIN(select Country, sum(sales) as total_sales_discount
        from orders inner join orders_break_down
        on orders.order_id = orders_break_down.order_id
        where discount <= 0
        group by Country)b
        ON a.Country=b.Country
        group by Country
        order a.Country Desc
        ;
        ': near "(": syntax error

#### 11.A Select Total Quantity, Total Sales for Each Country

In [121]:
# Select Total Quantity, Total Sales for Each Country
query = """
            select Country, sum(sales) as total_sales, sum(quantity) as total_quantity
            from orders inner join orders_break_down
            on orders.order_id = orders_break_down.order_id            
            group by Country
            order by total_sales, total_quantity desc
        ;
        """
salestotal= sql.read_sql(query, con=db_connection)
salestotal

,country,total_sales,total_quantity
0,Denmark,7763.0,204
1,Portugal,15106.0,286
2,Ireland,15998.0,392
3,Norway,20529.0,261
4,Finland,20702.0,201
5,Switzerland,24874.0,308
6,Sweden,30490.0,753
7,Belgium,42320.0,532
8,Netherlands,70313.0,1526
9,Austria,79382.0,973


#### 11.B Select Top 3 Country Based on Sales

In [117]:
# Select Top 3 Country Based on Sales
query = """
            SELECT Country, sum(sales) as total_sales
            from orders inner join orders_break_down
            on orders.order_id = orders_break_down.order_id
            group by Country
            order by total_sales Desc
            LIMIT 3
                    ;
        """
top3= sql.read_sql(query, con=db_connection)
top3

,country,total_sales
0,France,609683.0
1,Germany,488681.0
2,United Kingdom,420497.0


#### 11.C Select Bottom 3 Country Based On Quantities

In [120]:
# Select Bottom 3 Country Based On Quantities
query = """
            SELECT Country, sum(sales) as total_sales
            from orders inner join orders_break_down
            on orders.order_id = orders_break_down.order_id
            group by Country
            order by total_sales ASC
            LIMIT 3
                    ;
        """
BOTTOM3= sql.read_sql(query, con=db_connection)
BOTTOM3

,country,total_sales
0,Denmark,7763.0
1,Portugal,15106.0
2,Ireland,15998.0


#### 12. Select Average Sales By Categroy For The Country 'France'

In [142]:
# Select Average Sales By Categroy For The Country 'France'
query = """
            SELECT Category, country, avg(sales) as Average_sales
            from orders inner join orders_break_down
            on orders.order_id = orders_break_down.order_id
            where country = 'France'
            group by Category
            order by Average_sales ASC
            ;
        """
france= sql.read_sql(query, con=db_connection)
france

,category,country,Average_sales
0,Office Supplies,France,167.217709
1,Furniture,France,582.314381
2,Technology,France,595.145078


In [149]:
query = """
            SELECT country, category, avg(sales) as Average_sales
            from orders inner join orders_break_down
            on orders.order_id = orders_break_down.order_id
            group by country, category
            order by country, Average_sales ASC
            ;
        """
averagemax= sql.read_sql(query, con=db_connection)
averagemax

,country,category,Average_sales
0,Austria,Office Supplies,176.299435
1,Austria,Furniture,527.275000
2,Austria,Technology,576.297872
3,Belgium,Office Supplies,206.864583
4,Belgium,Furniture,443.500000
5,Belgium,Technology,715.315789
6,Denmark,Office Supplies,94.461538
7,Denmark,Furniture,163.444444
8,Denmark,Technology,217.333333
9,Finland,Office Supplies,160.526316


#### 13. Select Country, Category and Total Sales Where Average Total Sales is The Highest

In [161]:
query = """
        WITH average AS (
        SELECT country,Category, avg(sales) as Average_sales
        from orders inner join orders_break_down
        on orders.order_id = orders_break_down.order_id       
        group by country, Category
        )
        select country, category, max(Average_sales) max_average
        from average     
        group by country
        order by max_average
        ;
        """
averagemax= sql.read_sql(query, con=db_connection)
averagemax

,country,Category,max_average
0,Denmark,Technology,217.333333
1,Sweden,Furniture,305.055556
2,Ireland,Technology,333.714286
3,Netherlands,Furniture,372.263158
4,Portugal,Technology,379.230769
5,Italy,Technology,478.229947
6,Austria,Technology,576.297872
7,France,Technology,595.145078
8,Finland,Furniture,600.125000
9,Germany,Technology,627.913183


### JOINS

Below is a link to a handy reference for SQL joins. In this chart joins are represented in terms of sets and venn diagrams. 
https://www.codeproject.com/Articles/33052/Visual-Representation-of-SQL-Joins

Alternatively, remember the merge functionality of pandas.
https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf

### ADDITIONAL RESOURCES

- [sqlite3 home](http://www.sqlite.org)  
- [sqlite3 Python documentation](https://docs.python.org/3/library/sqlite3.html)
- [SQLite Python tutorial](http://sebastianraschka.com/Articles/2014_sqlite_in_python_tutorial.html)  
- [SQL zoo](http://www.sqlzoo.net)  Great for learning syntax



---



---



> > > > > > > > > © 2019 Institute of Data


---



---



